# Assistants

[Assistants](https://docs.langchain.com/langsmith/assistants) give developers a quick and easy way to create versioned agent configurations for experimentation.

## Supplying configuration to the graph

`task_maistro` is already set up to use assistants via `configuration.py`.

The following configurable fields are available and are loaded inside each graph node:

| Field | Default | Purpose |
|---|---|---|
| `user_id` | `"default-user"` | Namespaces all memories per user |
| `todo_category` | `"general"` | Separates todo lists (e.g. `personal`, `work`) |
| `task_maistro_role` | task management assistant | System prompt / role for the agent |

## Creating assistants

A practical use case for assistants is having **separate ToDo lists for different categories** — for example, one for personal tasks and one for work tasks.

Each assistant uses the same graph but gets its own isolated memory namespace via `todo_category`, and its own behavior via `task_maistro_role`.

In [1]:
%%capture --no-stderr
%pip install -U langgraph_sdk

This is the default assistant that we created when we deployed the graph.

In [4]:
from langgraph_sdk import get_client
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

### Personal assistant

This is the personal assistant that I'll use to manage my personal tasks.

In [3]:
personal_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro", 
    config={"configurable": {"todo_category": "personal"}}
)
print(personal_assistant)

{'assistant_id': '33fde19b-6cc6-41e2-ba73-56c1ee08dfe4', 'graph_id': 'task_maistro', 'version': 1, 'created_at': '2026-02-17T05:19:23.522186+00:00', 'updated_at': '2026-02-17T05:19:23.522186+00:00', 'config': {'configurable': {'todo_category': 'personal'}}, 'context': {'todo_category': 'personal'}, 'metadata': {}, 'name': 'Untitled', 'description': None}


Let's update this assistant to include my `user_id` for convenience,  [creating a new version of it](https://docs.langchain.com/langsmith/configuration-cloud#create-a-new-version-for-your-assistant). 

In [4]:
task_maistro_role = """You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:

- Help track and organize personal tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- Proactively ask for deadlines when new tasks are added without them
- Maintain a supportive tone while helping the user stay accountable
- Help prioritize tasks based on deadlines and importance

Your communication style should be encouraging and helpful, never judgmental. 

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Would you like to add one to help us track it better?"""

configurations = {
    "todo_category": "personal",
    "user_id": "pytholic",
    "task_maistro_role": task_maistro_role,
}

personal_assistant = await client.assistants.update(
    personal_assistant["assistant_id"], config={"configurable": configurations}
)
print(personal_assistant)

{'assistant_id': '33fde19b-6cc6-41e2-ba73-56c1ee08dfe4', 'graph_id': 'task_maistro', 'version': 2, 'created_at': '2026-02-17T05:19:23.522186+00:00', 'updated_at': '2026-02-17T05:19:23.537000+00:00', 'config': {'configurable': {'task_maistro_role': 'You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:\n\n- Help track and organize personal tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- Proactively ask for deadlines when new tasks are added without them\n- Maintain a supportive tone while helping the user stay accountable\n- Help prioritize tasks based on deadlines and importance\n\nYour communication style should be encouraging and helpful, never judgmental. \n\

### Work assistant

Now, let's create a work assistant. I'll use this for my work tasks.

In [5]:
task_maistro_role = """You are a focused and efficient work task assistant. 

Your main focus is helping users manage their work commitments with realistic timeframes. 

Specifically:

- Help track and organize work tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:
  • Developer Relations features: typically 1 day
  • Course lesson reviews/feedback: typically 2 days
  • Documentation sprints: typically 3 days
- Help prioritize tasks based on deadlines and team dependencies
- Maintain a professional tone while helping the user stay accountable

Your communication style should be supportive but practical. 

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Based on similar tasks, this might take [suggested timeframe]. Would you like to set a deadline with this in mind?"""

configurations = {
    "todo_category": "work",
    "user_id": "pytholic",
    "task_maistro_role": task_maistro_role,
}

work_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro",
    config={"configurable": configurations},
)
print(work_assistant)

{'assistant_id': 'f6d027a0-b7c6-4539-9640-e5704a108f3c', 'graph_id': 'task_maistro', 'version': 1, 'created_at': '2026-02-17T05:19:23.552832+00:00', 'updated_at': '2026-02-17T05:19:23.552832+00:00', 'config': {'configurable': {'task_maistro_role': 'You are a focused and efficient work task assistant. \n\nYour main focus is helping users manage their work commitments with realistic timeframes. \n\nSpecifically:\n\n- Help track and organize work tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n  • Developer Relations features: typically 1 day\n  • Course lesson reviews/feedback: typically 2 days\n  • Documentation sprints: typically 3 days\n- Help prioritize tasks base

## Using assistants 

Assistants will be saved to `Postgres` in our deployment.  

This allows us to easily search <!--[~search~](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/)--> [search](https://reference.langchain.com/python/langsmith/deployment/sdk/#langgraph_sdk.client.AssistantsClient.search) for assistants with the SDK.

In [6]:
assistants = await client.assistants.search()
for assistant in assistants:
    print({
        'assistant_id': assistant['assistant_id'],
        'version': assistant['version'],
        'config': assistant['config']
    })

{'assistant_id': 'f6d027a0-b7c6-4539-9640-e5704a108f3c', 'version': 1, 'config': {'configurable': {'task_maistro_role': 'You are a focused and efficient work task assistant. \n\nYour main focus is helping users manage their work commitments with realistic timeframes. \n\nSpecifically:\n\n- Help track and organize work tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n  • Developer Relations features: typically 1 day\n  • Course lesson reviews/feedback: typically 2 days\n  • Documentation sprints: typically 3 days\n- Help prioritize tasks based on deadlines and team dependencies\n- Maintain a professional tone while helping the user stay accountable\n\nYour communicati

We can manage them easily with the SDK. For example, we can delete assistants that we're no longer using.  

In [7]:
# create a temporary assitant
temp_assistant = await client.assistants.create(
    "task_maistro", 
    config={"configurable": configurations}
)

assistants = await client.assistants.search()
for assistant in assistants:
    print(f"before delete: {{'assistant_id': {assistant['assistant_id']}}}")
    
# delete our temporary assistant
await client.assistants.delete(assistants[-1]["assistant_id"])
print()

assistants = await client.assistants.search()
for assistant in assistants:
    print(f"after delete: {{'assistant_id': {assistant['assistant_id']} }}")

before delete: {'assistant_id': 9c60087b-5686-4c79-93e3-68c62e214ed4}
before delete: {'assistant_id': f6d027a0-b7c6-4539-9640-e5704a108f3c}
before delete: {'assistant_id': 33fde19b-6cc6-41e2-ba73-56c1ee08dfe4}
before delete: {'assistant_id': a4ca5828-5365-4a0a-95d3-cd08a81fa9f9}
before delete: {'assistant_id': e9a29978-fa17-4afe-bc5a-270ed9856910}
before delete: {'assistant_id': de0877c0-5b26-4e44-8ad1-7dae62a03167}

after delete: {'assistant_id': 9c60087b-5686-4c79-93e3-68c62e214ed4 }
after delete: {'assistant_id': f6d027a0-b7c6-4539-9640-e5704a108f3c }
after delete: {'assistant_id': 33fde19b-6cc6-41e2-ba73-56c1ee08dfe4 }
after delete: {'assistant_id': a4ca5828-5365-4a0a-95d3-cd08a81fa9f9 }
after delete: {'assistant_id': e9a29978-fa17-4afe-bc5a-270ed9856910 }


Let's set the assistant IDs for the `personal` and `work` assistants that I'll work with.

In [8]:
work_assistant_id = work_assistant['assistant_id']
personal_assistant_id = personal_assistant['assistant_id']

### Work assistant

Let's add some ToDos for my work assistant.

In [9]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

user_input = "Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_lIqhuYzi7wMhVRyAlaWJI7YE)
 Call ID: call_lIqhuYzi7wMhVRyAlaWJI7YE
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Re-film Module 6, lesson 5', 'time_to_complete': 120, 'deadline': '2026-02-17T23:59:59', 'status': 'not started'}
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_3dYRSzECfqoyyF1Gcnbn9PLQ)
 Call ID: call_3dYRSzECfqoyyF1Gcnbn9PLQ
  Args:
    update_type: todo
================================= Tool Message =================================

Document 480f03fa-fb4c-4199-8aa3-5d7d98690a17 updated:
Plan: Update th

In [10]:
user_input = "Create another ToDo: Finalize set of report generation tutorials."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create another ToDo: Finalize set of report generation tutorials.
================================== Ai Message ==================================

I notice that the task "Finalize set of report generation tutorials" doesn't have a deadline yet. Based on similar tasks, this might take around 2 days. Would you like to set a deadline with this in mind?


The assistant uses it's instructions to push back with task creation! 

It asks me to specify a deadline :) 

In [11]:
user_input = "OK, for this task let's get it done by next Tuesday."
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

OK, for this task let's get it done by next Tuesday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_4o5cpR0scXtMlvGa0IX0eH1E)
 Call ID: call_4o5cpR0scXtMlvGa0IX0eH1E
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Finalize set of report generation tutorials', 'time_to_complete': 120, 'deadline': '2026-02-21T23:59:59', 'solutions': [], 'status': 'not started'}

Document c5e9f1fa-6aed-473a-916a-096c6cbeb6ab updated:
Plan: Update the existing ToDo list with the new task for report generation tutorials.
Added content: {'task': 'Finalize set of report generation tutorials', 'status': 'not started', 'deadline': '2026-02-21T23:59:59', 'solutions': [], 'time_to_complete': 120}
================================== Ai Message ==========================

### Personal assistant

Similarly, we can add ToDos for my personal assistant.

In [12]:
user_input = "Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_8cMoYjVw0JzmHC3mnHgPUIKd)
 Call ID: call_8cMoYjVw0JzmHC3mnHgPUIKd
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Check on swim lessons for the baby this weekend.', 'time_to_complete': 30, 'deadline': '2026-02-19T17:00:00', 'status': 'not started', 'solutions': ['Contact local swim schools', 'Check online for swim lesson availability']}
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_qwxG8MC3BrXVCchqGIAarN0L)
 Call ID: call_qwxG8MC3BrXVCchqGIAarN0L
  Args:
    update_type: todo
================================= Tool Message 

In [13]:
user_input = "Give me a todo summary."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Give me a todo summary.
================================== Ai Message ==================================

Here's your current todo summary:

### Overdue Tasks
- None

### Tasks Due Today
- None

### Tasks Due This Week
- None

### Future Tasks
1. **Check on swim lessons for the baby this weekend.**
   - **Deadline:** 2026-02-19T17:00:00
   - **Status:** Not started
   - **Solutions:** 
     - Contact local swim schools
     - Check online for swim lesson availability
   - **Time to Complete:** 30 minutes

2. **For winter travel, check AmEx points.**
   - **Deadline:** None
   - **Status:** Not started
   - **Time to Complete:** 15 minutes

I notice that the task "For winter travel, check AmEx points." doesn't have a deadline yet. Would you like to add one to help us track it better? 

Also, if you have any other tasks or updates, feel free to share!


Example response from the interactive API docs after adding tasks to both assistants:

```json
{
  "items": [
    {
      "namespace": ["todo", "personal", "alice"],
      "key": "c535a734-58b6-4f22-830c-058dce161ea3",
      "value": {
        "task": "Book flights for the upcoming trip.",
        "status": "not started",
        "deadline": null,
        "solutions": ["Check Google Flights", "Compare with Skyscanner"],
        "time_to_complete": 20
      },
      "created_at": "2026-02-17T05:19:51.519993+00:00",
      "updated_at": "2026-02-17T05:19:51.519993+00:00",
      "score": null
    },
    {
      "namespace": ["todo", "personal", "alice"],
      "key": "9df1b632-5cc7-4f52-acfd-bfd0d71f83cb",
      "value": {
        "task": "Schedule dentist appointment.",
        "status": "not started",
        "deadline": "2026-02-28T17:00:00",
        "solutions": ["Call the clinic", "Book online via their portal"],
        "time_to_complete": 15
      },
      "created_at": "2026-02-17T05:19:51.518189+00:00",
      "updated_at": "2026-02-17T05:19:51.518189+00:00",
      "score": null
    },
    {
      "namespace": ["todo", "work", "alice"],
      "key": "c5e9f1fa-6aed-473a-916a-096c6cbeb6ab",
      "value": {
        "task": "Prepare Q1 project status update.",
        "status": "not started",
        "deadline": "2026-02-21T23:59:59",
        "solutions": ["Pull metrics from dashboard", "Draft slides"],
        "time_to_complete": 90
      },
      "created_at": "2026-02-17T05:19:35.400025+00:00",
      "updated_at": "2026-02-17T05:19:45.054373+00:00",
      "score": null
    },
    {
      "namespace": ["todo", "work", "alice"],
      "key": "480f03fa-fb4c-4199-8aa3-5d7d98690a17",
      "value": {
        "task": "Review open pull requests.",
        "status": "not started",
        "deadline": "2026-02-19T17:00:00",
        "solutions": ["Check GitHub notifications", "Prioritize by age"],
        "time_to_complete": 45
      },
      "created_at": "2026-02-17T05:19:26.327340+00:00",
      "updated_at": "2026-02-17T05:19:35.397450+00:00",
      "score": null
    }
  ]
}
```


## Cleanup store

In [6]:
# Delete all todos for a specific namespace
items = await client.store.search_items(("todo", "personal", "default-user"), limit=100)
for item in items["items"]:
    await client.store.delete_item(("todo", "personal", "default-user"), key=item["key"])

# # Do the same for personal if needed
# items = await client.store.search_items(("todo", "general", "pytholic"), limit=100)
# for item in items["items"]:
#     await client.store.delete_item(("todo", "general", "pytholic"), key=item["key"])

# # Delete all todos for general namespace
# items = await client.store.search_items(("todo", "general", "Test-Double-Texting"), limit=100)
# for item in items["items"]:
#     await client.store.delete_item(("todo", "general", "Test-Double-Texting"), key=item["key"])
